In [1]:
import pandas as pd
import scipy as sp
import numpy as np
import os
from hd_var import *
from functools import partial

In [39]:
# Load Data

# Specify names of Data and Mapping subfolders

data_dir = 'data'
map_dir = 'mapping'

# Create a adictionary to fix Direct & Fund asset names

agg_dict = pd.read_excel(f'{map_dir}\\name_mapping.xlsx')
agg_dict.set_index(agg_dict.columns[0], drop = True, inplace = True)
agg_dict = agg_dict.to_dict()['New Name']

# Load the Sector Mapping Excel file and create dictionaries to map any individual asset to its corresponding
# Sector, Region, Country, and Investment Type

map_df =  pd.read_excel(f'{map_dir}\\sector-mapping-rm.xlsx', header = 0) 
map_df.set_index('Asset Name', inplace = True)
map_df.index = [agg_dict.get(i) if i in agg_dict else i for i in map_df.index]

# Define different dictionaries using asset names as keys

map_dict = map_df.to_dict()

sector_dict = map_dict['Sector']
region_dict = map_dict['Region']
country_dict = map_dict['Country']
type_dict = map_dict['Investment Type']

# Define which sectors belong to Direct and which to Fund (Static)

direct_sectors = [
    'Electric & Gas Utilities',
    'Agriculture',
    'Telecommunication',
    'Water & WW Utilities',
    'Transportation',
    'Timber',
    'Multi-Utilities',
    'Renewable Power',
    'Other',
    'Total'
]

fund_sectors = [
    'Agriculture (Funds)',
    'Energy',
    'General Infrastructure',
    'Timber (Funds)',
    'Total'
]

In [40]:
# Import Exposure file

expo_file = pd.ExcelFile(f"{data_dir}\\Exposure - September 30, 2022_v4.xlsx")

# Read Market Value in $CAD

mv_df = pd.read_excel(
    io = expo_file, 
    sheet_name = "Market Values",
    skiprows = range(3),
    usecols = "B, I")

mv_df.columns = ['Asset Name', 'MV CAD']        
mv_df.set_index("Asset Name", inplace = True)
drop_list = list(mv_df[mv_df.index.str.contains('Total')].index) + ['IFT']
mv_df.drop(axis=0, index = drop_list, inplace = True) # drop subtotals

# Add Country, Region, and Sector in mv_df

mv_df['Country'] = [country_dict.get(i) for i in mv_df.index]
mv_df['Region'] = [region_dict.get(i) for i in mv_df.index]
mv_df['Sector'] = [sector_dict.get(i) for i in mv_df.index]

# Read IFT CAD market values on the asset level

IFT_df = pd.read_excel(
    io = expo_file, 
    sheet_name = "Program Exposure",
    skiprows = range(6),
    usecols = "B, C, K") # Investment Code, Investment Name, MV Base

IFT_df = IFT_df[IFT_df['Investment Name'] == 'IFT']

# IFT_df['Region'] = [region_dict.get(i) for i in IFT_df['Investment Code']]
# IFT_df['Country'] = [country_dict.get(i) for i in IFT_df['Investment Code']]
IFT_df.columns = ['Asset Name', 'Investment Name', 'MV CAD']
IFT_df.set_index('Asset Name', inplace = True)
IFT_df.drop('Investment Name', axis = 1, inplace = True)

# Append MV with IFT, netting the MV

mv_df['MV CAD Net IFT'] = mv_df['MV CAD'].add(IFT_df['MV CAD'], fill_value=0)

c:\Users\miwang\Anaconda3\envs\env_full\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [41]:
# Generate a list of csv files for the script to read from

tree = [i for i in os.walk(data_dir)]
lst = []

for i in range(1, len(tree)):
    for j, item in enumerate(tree[i][2]):
        file = f'{tree[i][0]}\\{item}'
        # set a lower limit of file size to filter out the settings & summary reports
        if os.stat(file).st_size > 2048:
            lst.append(file)
            
df_names = [i.split('\\')[-2] for i in lst]
# print to verify which RM reports we are looking at based on sub-director names
print(df_names)

# "dfs" is a list that stores all dataframes created using the csv files
# the order in the list corresponds to the order of df_names

dfs = []
for i, x in enumerate(lst):
    dfs.append(pd.read_csv(lst[i], thousands=','))

['Infra Dashboard_PnL Report_Country_v1.csv', 'Infra Dashboard_PnL Report_Direct_Fund.csv', 'Infra dashboard_PV Report_Country.csv', 'Infra Dashboard_PV Report_Direct_Fund.csv']


In [42]:
# Clean up dataframes' column labels to make them more readable

col_names = [[] for _ in range(len(dfs))]

for i, df in enumerate(dfs):
    temp = df.columns.to_list()
    col_names[i].append(temp[0]) # append 'level' since it doesn't have brackets
    for j in temp[1:]:
        label_short = j[j.rfind("[") + 1:j.rfind("]")]
        col_names[i].append(label_short)
        
for i, df in enumerate(dfs):
    df.columns = col_names[i]
    df.drop('level', inplace = True, axis = 1)

In [43]:
# Create six DataFrames:
# fund_pv_df, fund_pnl_df
# direct_pv_df, direct_pnl_df
# country_pv_df, country_pnl_df

fund_pnl_df = dfs[1].filter(
    items = [col for col in dfs[1].columns if 'Fund' in col]
)

direct_pnl_df = dfs[1].filter(
    items = [col for col in dfs[1].columns if 'Direct' in col]
)

country_pnl_df = dfs[0].filter(
    items = [col for col in dfs[0].columns if 'Unspecified' not in col]
)

fund_pv_df = dfs[2].loc[
    dfs[2]['acInvestmentType']=='Fund', ['fundName','PV']
].groupby('fundName').sum()

direct_pv_df = dfs[2].loc[
    dfs[2]['acInvestmentType']=='Direct', ['fundName','PV']
].groupby('fundName').sum()

country_pv_df = dfs[2][['BCI_country','PV']].groupby('BCI_country').sum()


# Add MV to fund_pv_df, direct_pv_df, country_pv

for df in [fund_pv_df, direct_pv_df]:
    df.index = [agg_dict.get(i) if i in agg_dict else i for i in df.index]

fund_pv_df = fund_pv_df.join(mv_df, how = 'left')
direct_pv_df = direct_pv_df.join(mv_df, how = 'left')
country_pv_df = country_pv_df.join(mv_df.groupby(by = 'Country').sum(), how = 'outer')

# Get scenarios (dates in time series)

scenario = dfs[0]['Scenario'].to_list()

# re-construct DataFrame index

for df in [fund_pnl_df, direct_pnl_df]:
    if 'Scenario' in df.columns:
        df.drop('Scenario', axis = 1, inplace = True)
    df.columns = [i.split('\\')[-1] for i in df.columns]
    df.index = scenario
    
for df in [country_pnl_df]:
    if 'Scenario' in df.columns:
        df.drop(['Scenario'] , axis = 1, inplace = True)
    df.columns = [i.split('\\')[-2] for i in df.columns]
    df.index = scenario
    
country_pnl_df.drop(['Total', 'Derivatives'], axis = 1, inplace = True)
country_pv_df.drop(['*Unspecified', 'Derivatives', 'Global'], axis = 0, inplace = True)

C:\Users\miwang\AppData\Local\Temp\1\ipykernel_5616\837416913.py:36: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_pv_df = country_pv_df.join(mv_df.groupby(by = 'Country').sum(), how = 'outer')


In [44]:
# Constant PV numbers

tot_pv = dfs[2].loc[0,'PV']
fund_pv = fund_pv_df['PV'].sum()
direct_pv = direct_pv_df['PV'].sum()

# Cosntant MV numbers

tot_mv = mv_df['MV CAD Net IFT'].sum()
fund_mv = fund_pv_df['MV CAD Net IFT'].sum()
direct_mv = direct_pv_df['MV CAD Net IFT'].sum()

# Total IRR PnL as a list to pass into percentage VaR calculations

tot_pnl = dfs[1]['PNL\Total\Total'].to_list()

In [45]:
country_pv_df

,PV,MV CAD,MV CAD Net IFT
AE,2.607480e+07,1.674495e+08,8.194366e+06
AT,2.362583e+07,NaN,NaN
AU,1.727189e+09,1.513267e+09,1.513267e+09
BR,7.579816e+08,6.908001e+08,6.701654e+08
CA,2.724026e+09,4.309711e+09,4.309711e+09
CL,1.662842e+09,1.701683e+09,1.701683e+09
CN,5.685132e+06,NaN,NaN
CO,1.019045e+09,8.993192e+08,8.993192e+08
CZ,6.838679e+08,6.458073e+08,6.458073e+08
DE,1.000108e+09,9.009294e+08,9.009294e+08


In [46]:
# VaR Calculation by Sector by Investment Type
#
# from pyinstrument import Profiler
# profiler = Profiler()
# profiler.start()

total_result = pd.DataFrame(
    {
        'Asset Name': ['I&RR Total'],
        'PV': [tot_pv],
        'MV CAD Net of IFT': [tot_mv],
        'VaR': [hd_var_ann(tot_pnl, tot_pv)],
        'Risk Contribution to I&RR': [1]
    }
)

for inv_type in ['fund', 'direct']:

    sectors = vars()[f'{inv_type}_sectors'] # i.e., fund_sectors or direct_sectors
    
    pv = []
    mv = []
    var = []
    contrib = []
    var_lst =[]
    
    pnl_df = vars()[f'{inv_type}_pnl_df'] # i.e., fund_pnl_df or direct_pnl_df
    pv_df = vars()[f'{inv_type}_pv_df'] # i.e., fund_pv_df or direct_pv_df
    
    pv_lst = [pv_df['PV'].sum()] + pv_df['PV'].to_list() # add total PV in the beginning of the PV list

    mv_lst = [pv_df['MV CAD Net IFT'].sum()] + pv_df['MV CAD Net IFT'].to_list()
    
    var_lst = [hd_var_ann(pnl_df.iloc[:, i], x, ci=0.95, factor=25.2**0.5) 
               for i, x in enumerate(pv_lst)]
    contrib_lst = [hd_contrib(tot_pnl, pnl_df.iloc[:, i]) 
                   for i, _ in enumerate(pv_lst)]
    
    var_df = pd.DataFrame(
        {
        'Asset Name': list(pnl_df),
        'PV': pv_lst,
        'MV CAD Net of IFT': mv_lst,
        'VaR': var_lst,
        'Risk Contribution to I&RR': contrib_lst
        }
    )
    
    vars()[f'{inv_type}_var_df'] = var_df.sort_values(by = 'VaR', axis=0, ascending = False)
    vars()[f'{inv_type}_var_df'].reset_index(drop = True, inplace = True)
    
    for i, sect in enumerate(sectors):      
        # iterates through all sectors in Funds
        if sect == 'Total':
            assets = ['Total']
            sector_pv = vars()[f'{inv_type}_pv']
            sector_mv = vars()[f'{inv_type}_mv']
        else:
            assets = [i for i in list(pnl_df) if sector_dict.get(i) == sect]
            # list of PNL columns names that fall into the sector
            sector_pv = pv_df[pv_df['Sector'] == sect]['PV'].sum(axis=0)
            sector_mv = pv_df[pv_df['Sector'] == sect]['MV CAD Net IFT'].sum(axis=0)
        # print(f'{sect}: {funds_in_sect}')
        # print(funds_in_sect)
        pnl = pnl_df[assets].sum(axis = 1) # this is a PnL list for a certain sector
        pv.append(sector_pv)
        mv.append(sector_mv)
        var.append(hd_var_ann(pnl, sector_pv))
        contrib.append(hd_contrib(tot_pnl, pnl))
    
    result = pd.DataFrame({'Sub-Industry': sectors, 'PV': pv, 'MV CAD Net of IFT': mv, 'VaR': var, 'Risk Contribution to I&RR': contrib})
    
    vars()[f'{inv_type}_result'] = result
    
# profiler.stop()
# profiler.print()

In [47]:
# calculate COUNTRY specific VaR and VaR contribution
countries = list(country_pnl_df)
country_pv = country_pv_df['PV'].to_list()
country_mv = country_pv_df['MV CAD Net IFT'].to_list()
country_var = [hd_var_ann(country_pnl_df.iloc[:, i], x, ci=0.95, factor=25.2**0.5) 
               for i, x in enumerate(country_pv)] 
country_contrib = [hd_contrib(tot_pnl, country_pnl_df.iloc[:, i]) 
                   for i, _ in enumerate(country_pv)]
country_result = pd.DataFrame({'Country': countries, 
                               'PV': country_pv, 
                               'MV CAD Net IFT': country_mv,
                               'VaR': country_var, 
                               'Risk Contribution to I&RR': country_contrib})

In [48]:
# calculate REGION specific VaR and VaR contribution
region_pv_df = pd.concat([direct_pv_df, fund_pv_df], axis = 0)
region_pv_df.set_index('Region', inplace = True)
region_pv_df = region_pv_df.groupby('Region').sum()

region_pnl_df = pd.concat([direct_pnl_df.iloc[:, 1:], fund_pnl_df.iloc[:, 1:]], axis = 1)
region_pnl_df = region_pnl_df.rename(columns = agg_dict)
region_pnl_df = region_pnl_df.rename(columns = region_dict)
region_pnl_df = region_pnl_df.groupby(by = region_pnl_df.columns, axis = 1).sum()

regions = list(region_pnl_df)
region_pv = region_pv_df['PV'].to_list()
region_mv = region_pv_df['MV CAD Net IFT'].to_list()
region_var = [hd_var_ann(region_pnl_df.iloc[:, i], x, ci=0.95, factor=25.2**0.5) 
               for i, x in enumerate(region_pv)] 
region_contrib = [hd_contrib(tot_pnl, region_pnl_df.iloc[:, i]) 
                   for i, _ in enumerate(region_pv)]

region_result = pd.DataFrame({'Region': regions, 
                               'PV': region_pv, 
                               'MV CAD Net IFT': region_mv,
                               'VaR': region_var, 
                               'Risk Contribution to I&RR': region_contrib})

C:\Users\miwang\AppData\Local\Temp\1\ipykernel_5616\3114040059.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  region_pv_df = region_pv_df.groupby('Region').sum()


In [ ]:
results = ['total_result', 'fund_result', 'direct_result', 'fund_var_df', 'direct_var_df', 'country_result', 'region_result']

with pd.ExcelWriter(f'IRR_VaR_Results.xlsx') as writer:  
    for name in results:
        df = vars()[name]
        df.set_index(df.columns[0], drop = True, inplace = True)
        df.to_excel(writer, sheet_name = name)